<a href="https://colab.research.google.com/github/Bitachi/parking-detector-poc-ipynb/blob/main/parking_detector_poc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#必要なライブラリをインストール
!pip install ultralytics deep_sort_realtime opencv-python-headless --quiet



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.5 MB/s eta 0:00:00


In [3]:
#検証対象の動画ファイルおよびYOLOモデルをダウンロード
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/fixedPointCamera.mp4" -O /content/fixedPointCamera.mp4
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/yolov8l.pt" -O /content/yolov8l.pt
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/yolov8m.pt" -O /content/yolov8m.pt
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/yolov8n.pt" -O /content/yolov8n.pt
!wget "https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/yolov5s.pt" -O /content/yolov5s.pt

--2025-05-26 12:56:51--  https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/fixedPointCamera.mp4
Resolving parking-detection-source.s3.ap-northeast-1.amazonaws.com (parking-detection-source.s3.ap-northeast-1.amazonaws.com)... 3.5.158.28, 3.5.155.163, 52.219.199.38, ...
Connecting to parking-detection-source.s3.ap-northeast-1.amazonaws.com (parking-detection-source.s3.ap-northeast-1.amazonaws.com)|3.5.158.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 463496808 (442M) [video/mp4]
Saving to: ‘/content/fixedPointCamera.mp4’

/content/fixedPoint 100%[===================>] 442.02M  23.4MB/s    in 20s     

2025-05-26 12:57:12 (21.6 MB/s) - ‘/content/fixedPointCamera.mp4’ saved [463496808/463496808]

--2025-05-26 12:57:12--  https://parking-detection-source.s3.ap-northeast-1.amazonaws.com/yolov8l.pt
Resolving parking-detection-source.s3.ap-northeast-1.amazonaws.com (parking-detection-source.s3.ap-northeast-1.amazonaws.com)... 3.5.156.26, 3.5.154.10

In [5]:
#parking-detection-poc-yolov8m.py
import cv2
import os
import csv
import numpy as np
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict

# パス設定
VIDEO_PATH = "/content/fixedPointCamera.mp4"      # 入力動画
MODEL_PATH = "/content/yolov8m.pt"                # YOLOv8モデル
CSV_PATH = "/content/stopped_vehicles.csv"        # 出力CSV
OUTPUT_IMAGE_DIR = "/content/images/yolov8m"       # 出力画像ディレクトリ

# モデルロード
model = YOLO(MODEL_PATH)

# 出力フォルダ作成
os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)

# Deep SORT初期化
tracker = DeepSort(max_age=30)

# パラメータ
STOP_THRESHOLD_SECONDS = 10
IOU_THRESHOLD = 0.5
MAX_SECONDS = 180

vehicle_states = defaultdict(lambda: {"positions": [], "is_stopped": False, "stop_start": None})
confirmed_stopped_vehicles = []

def compute_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1b, y1b, x2b, y2b = box2
    xi1, yi1 = max(x1, x1b), max(y1, y1b)
    xi2, yi2 = min(x2, x2b), min(y2, y2b)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2b - x1b) * (y2b - y1b)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration_seconds = int(total_frames / fps)

for current_sec in range(min(duration_seconds, MAX_SECONDS)):
    frame_number = int(current_sec * fps)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)[0]
    detections = []
    for box in results.boxes:
        cls_id = int(box.cls[0])
        if cls_id not in [2, 5, 7]:  # 車、バス、トラック
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cls_id))

    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue
        track_id = track.track_id
        ltrb = track.to_ltrb()
        x1, y1, x2, y2 = map(int, ltrb)
        box = (x1, y1, x2, y2)

        state = vehicle_states[track_id]
        state["positions"].append(box)

        if len(state["positions"]) >= STOP_THRESHOLD_SECONDS:
            recent_positions = state["positions"][-STOP_THRESHOLD_SECONDS:]
            ious = [compute_iou(recent_positions[0], pos) for pos in recent_positions[1:]]
            if all(iou > IOU_THRESHOLD for iou in ious):
                if not state["is_stopped"]:
                    state["is_stopped"] = True
                    state["stop_start"] = current_sec
            else:
                if state["is_stopped"]:
                    confirmed_stopped_vehicles.append({
                        "vehicle_id": track_id,
                        "start_time": state["stop_start"],
                        "end_time": current_sec
                    })
                    state["is_stopped"] = False
                    state["stop_start"] = None

        color = (0, 0, 255) if state["is_stopped"] else (255, 0, 0)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 画像保存
    filename = os.path.join(OUTPUT_IMAGE_DIR, f"frame_{current_sec:04d}.jpg")
    cv2.imwrite(filename, frame)

# 終了時にまだ停止している車両も記録
for track_id, state in vehicle_states.items():
    if state["is_stopped"]:
        confirmed_stopped_vehicles.append({
            "vehicle_id": track_id,
            "start_time": state["stop_start"],
            "end_time": MAX_SECONDS
        })

# CSV書き出し
with open(CSV_PATH, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Vehicle_ID", "Start_Second", "End_Second"])
    for vehicle in confirmed_stopped_vehicles:
        writer.writerow([
            vehicle["vehicle_id"],
            vehicle["start_time"],
            vehicle["end_time"]
        ])

cap.release()
print("処理完了。1秒ごとのフレームで停止車両を検出しました。")



0: 384x640 4 persons, 2 cars, 1003.2ms
Speed: 2.4ms preprocess, 1003.2ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 3 cars, 917.0ms
Speed: 2.9ms preprocess, 917.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 919.5ms
Speed: 3.1ms preprocess, 919.5ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 2 cars, 1438.2ms
Speed: 3.2ms preprocess, 1438.2ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 3 cars, 1108.4ms
Speed: 2.4ms preprocess, 1108.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 3 cars, 1094.8ms
Speed: 3.1ms preprocess, 1094.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 5 cars, 1062.7ms
Speed: 3.0ms preprocess, 1062.7ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 persons, 4 cars, 

In [1]:
#ディレクトリ配下の画像を全削除する

import glob
import os

folder_path = "/content/images/yolov8m"
jpg_files = glob.glob(os.path.join(folder_path, "*.jpg"))

for file in jpg_files:
    os.remove(file)

print(f"✅ {len(jpg_files)} 枚の JPG ファイルを削除しました。")


✅ 0 枚の JPG ファイルを削除しました。
